# Queens Problem

### importem llibreries que utilitzarem i definim variables globals


In [2]:
import numpy as np
import random

INITIAL_POPULATION = 10
BOARD_SIZE = 8
MUTATION_CHANCE = 0.1

### Definir Agent, població i display

In [127]:
def init_agent(board_size):
    board = []
    pickedPositions = [y for y in range(0, board_size)]

    for i in range(board_size):
        choice = random.choice(pickedPositions)
        pickedPositions.remove(choice)
        board.append(choice)

    return board

def display_agent(agent):
    for queenPosition in agent:
        result = []

        for el in range(len(agent)):
            result.append(1 if el == queenPosition else 0)

        print(result)

def init_population(initial_population_size, board_size):
    return [init_agent(board_size) for _ in range(initial_population_size)]


_population = init_population(INITIAL_POPULATION, BOARD_SIZE)
_agent = _population[0]
assert BOARD_SIZE == len(_agent)
assert INITIAL_POPULATION == len(_population)
display_agent(_agent)
# assert BOARD_SIZE*BOARD_SIZE == len(display_agent(_agent))

[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0]


### definir crossover

In [154]:
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1))
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

child = crossover(_population[0], _population[1])
print(_population[0])
print(_population[1])
print(child)

[0, 5, 1, 2, 4, 3, 7, 6]
[4, 7, 6, 0, 5, 1, 3, 2]
[4, 7, 6, 0, 5, 1, 3, 2]


### definir mutació


In [167]:
def mutation(agent):
    newAgent = agent.copy()
    pos1, pos2 = random.sample(range(len(newAgent)), 2)
    newAgent[pos1], newAgent[pos2] = newAgent[pos2], newAgent[pos1]
    return newAgent

def mutation_with_chance(agent, mutation_chance):
    hasMutation = random.randrange(100) <= (mutation_chance * 100)

    if not hasMutation:
        return agent

    return mutation(agent)

print(mutation_with_chance(_agent, 1))
print(_agent)

[0, 5, 1, 2, 4, 3, 6, 7]
[0, 5, 1, 2, 4, 3, 7, 6]


### definir crossover de tota la població amb mutació

In [194]:
def replicate(population, mutation_chance):
    newPopulation = []

    for _ in range(len(population)):
        tournament = random.sample(population, 2)
        print(max(tournament, key=lambda x: x[1]))

replicate(_population, MUTATION_CHANCE)


[0, 6, 4, 3, 5, 1, 2, 7]
[0, 6, 4, 3, 5, 1, 2, 7]
[0, 6, 4, 3, 5, 1, 2, 7]
[7, 3, 0, 6, 4, 1, 5, 2]
[6, 7, 4, 3, 5, 1, 2, 0]
[0, 5, 1, 2, 4, 3, 7, 6]
[4, 7, 6, 0, 5, 1, 3, 2]
[4, 7, 6, 0, 5, 1, 3, 2]
[6, 1, 5, 3, 7, 4, 2, 0]
[4, 7, 6, 0, 5, 1, 3, 2]


### definir fitness function

mirar diagonals i treure repetides

In [116]:
def fitness_agent(agent):
    conflicts = 0
    for i in range(len(agent)):
        for j in range(i + 1, len(agent)):
            if abs(agent[i] - agent[j]) == j - i:
                conflicts += 1
    return conflicts

def fitness_population(population):
    pass

display_agent(_agent)
print(fitness_agent(_agent))
assert fitness_agent(_agent) <= BOARD_SIZE

[0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0]
3


### Population Selection (by score)


In [ ]:
def sort_population(population, fitness_list):
    pass

def select_population(population, fitness_list, population_size):
    pass



### definir main

(seguir els pasos de classe)

In [19]:
def main():
    # init
    
    # while
        # replicate
        # fitness
        # sort + select
        # new gen
        
    pass

### Provar el codi

In [ ]:
POPULATION_SIZE = 100
BOARD_SIZE = 20
MUTATION_CHANCE = 0.1

agent = main(POPULATION_SIZE, BOARD_SIZE, MUTATION_CHANCE)
display_agent(agent)